<a href="https://colab.research.google.com/github/Akshaychikara/NLP/blob/main/Chapter01_Project360.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:

from google.colab import files

#Importing the libraries for building Emotion Classifier
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
import string
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.util import ngrams
import re
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import sklearn.feature_extraction.text as text
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
import xgboost
from sklearn import decomposition, ensemble
import pandas, numpy, textblob, string
import re
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
#uploaded = files.upload()

Saving ISEAR.csv to ISEAR (1).csv


In [72]:
dataset = pd.read_csv("ISEAR.csv", header=None)
dataset.columns = ['EMOTION', 'TEXT','UNNAMED']
dataset = dataset[['EMOTION', 'TEXT']]
dataset.head()

,EMOTION,TEXT
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


In [79]:
dataset.info()
# There is no null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7516 entries, 0 to 7515
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   EMOTION  7516 non-null   object
 1   TEXT     7516 non-null   object
dtypes: object(2)
memory usage: 117.6+ KB


In [74]:
# Perfroming Preprocessing
# -------------------------
def preprocess(data):
  # Convert to lower case
  data['TEXT'] = data['TEXT'].apply(str.lower)

  # Remove stop words
  stop_words = stopwords.words('english')
  data['TEXT'] = data['TEXT'].apply(lambda sentense: " ".join(word for word in sentense.split() if word not in stop_words))

  # Correct spelling
  # Not applying str(TextBlob(a).correct()), becasuse it seems to produce incorect results.

  # Get root word
  st = PorterStemmer()
  data['TEXT'] = data['TEXT'].apply(lambda sentense: " ".join(st.stem(word) for word in sentense.split()))

  return data

processed_data = preprocess(dataset)
processed_data.head()

,EMOTION,TEXT
0,joy,day feel close partner friends. feel peac also...
1,fear,everi time imagin someon love could contact se...
2,anger,obvious unjustli treat possibl elucid this.
3,sadness,think short time live relat period life think ...
4,disgust,gather found involuntarili sit next two peopl ...


In [81]:
print(processed_data['EMOTION'].value_counts())
# It seems a well balanced data, except for one case where Emotion is "Guit". It seems misspelled. Will correct it manually.
print(processed_data[processed_data['EMOTION'] == "guit"])
processed_data['EMOTION'] = processed_data['EMOTION'].replace("guit", "guilt")
print("After correcrion..")
print(processed_data['EMOTION'].value_counts())

EMOTION
joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1049
guit          1
Name: count, dtype: int64

     EMOTION                                               TEXT
7499    guit  hostel, roommat selfish person would avoid any...
After correcrion..
EMOTION
joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: count, dtype: int64
